# Desafio - Aula 2

In [2]:
import pandas as pd
df = pd.read_csv("https://s3.amazonaws.com/automl-example/produtos.csv",
                 delimiter=";", encoding='utf-8')

### 1) utilizando o df acima carregado, faça:

   ### -> Elimine linhas com valores nulos
   ### -> Adicione uma nova coluna chamada texto, formada pela composição das colunas nome e descrição
   ### -> Quantos Unigramas existem antes e depois de remover stopwords
   ### -> Quantos Bigramas existem antes e depois de remover stopwords
   ### -> Quantos Trigramas existem antes e depois de remover stopwords
   ### -> Quantos verbos e Adverbios existem na nova coluna (utilize: from collections import Counter)
   ### -> Quantos unigramas existem após aplicar Stemmer (utilize rslp)

In [3]:
df.dropna(inplace=True)
df.isna().any()

nome         False
descricao    False
categoria    False
dtype: bool

In [4]:
df['texto'] = df['nome'] + ' ' + df['descricao']

In [11]:
from unidecode import unidecode
import re

def clearText(text):    
    ## Pontuação
    text = [re.sub('[^A-Za-z ]+', '', word) for word in text]
    ## Retirar numeros
    text = [re.sub('[0-9]+', '', word) for word in text]
    ## Retirar caracteres especiais e letras soltas
    text = [unidecode(name) for name in text if name and len(name) > 1]
    return text

In [12]:
import nltk
nltk.download('stopwords')
stops = nltk.corpus.stopwords.words('portuguese')
stops = clearText(stops)

[nltk_data] Downloading package stopwords to
[nltk_data]     /home/guilherme/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [22]:
from sklearn.feature_extraction.text import CountVectorizer

def getNgram(words, n):
    
    vect = CountVectorizer(ngram_range=(n, n))
    vect.fit(words)
    text_vect = vect.transform(words)

    print('Unigramas com stop words: ' + str(len(vect.get_feature_names())))
    # pd.DataFrame(text_vect.A, columns=names)

    ##########################################################################
    vect = CountVectorizer(ngram_range=(n, n), stop_words=stops)
    vect.fit(words)
    text_vect = vect.transform(words)

    print('Unigramas sem stop words: ' + str(len(vect.get_feature_names())))
    
    # pd.DataFrame(text_vect.A, columns=names)

    return vect.get_feature_names()


In [25]:
words = df.texto
words = clearText(words)

ngram = getNgram(words, 1)

Unigramas com stop words: 35604
Unigramas sem stop words: 35450


In [26]:
ngram = getNgram(words, 2)

Unigramas com stop words: 149715
Unigramas sem stop words: 135158


In [27]:
ngram = getNgram(words, 3)

Unigramas com stop words: 212541
Unigramas sem stop words: 161895


### 2) crie um tagger baseado em expressões regulares:
   ### -> crie uma variável que amazenará uma tupla (expressão, tag)
   ### -> o tagger deverá capturar gerúndio (ando, endo, indo), plurais e números cardinais
   ### -> utilize nltk.RegexpTagger(variável) para carregar seu tagger
   ### -> apresente uma frase teste para cada tipo de expressão

In [116]:
import re

tags = [
    (r'.+ando|endo|indo&', 'gerundio'),
    (r'.+s$', 'plural'),
    (r'[0-9]+', 'numeros')
]

In [117]:
import nltk
tagger = nltk.RegexpTagger(tags)

In [118]:
text = "Veja os 2 animais andando na rua"
tolk = nltk.word_tokenize(text)
tagger.tag(tolk)

[('Veja', None),
 ('os', 'plural'),
 ('2', 'numeros'),
 ('animais', 'plural'),
 ('andando', 'gerundio'),
 ('na', None),
 ('rua', None)]